In [59]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

### Top player winnings per year

In [66]:
#get details of players on each page
def pull_player_data(df_top_players, session, url, year):
    table = None
    
    while table is None:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find("table", class_ = 'detail_list_table')
    
    row_list = []
    
    tr_list = table.find('tbody')
    for tr in tr_list:
        row = tr.find_all('td')
        
        rank = row[0].text.strip()
        player_id = row[1].text.strip()
        player_name = row[2].text.strip()
        winnings_year = row[3].text.strip()
        winnings_overall = row[4].text.strip()
        pct_winnings_year = row[5].text.strip()
        player_profile = row[2].a['href']

        row_list.append([rank, player_id, player_name, winnings_year, winnings_overall, pct_winnings_year, year, player_profile])
        
    df_top_players = pd.concat([df_top_players, pd.DataFrame(row_list, columns=df_top_players.columns)])
    
    return df_top_players

In [159]:
#get the top 500 players per year for each year from 1998-2022
#500 players = 5 pages; 2001 has 4 pages, 2000 has 3 pages, etc.
def top_players_table(): 
    df_top_players = pd.DataFrame(columns=['rank', 'player_id', 'player_name', 'winnings_year', 'winnings_overall', 'pct_winnings_overall', 'year', 'player_profile'])

    for year in range(2022, 1997, -1):
        #get top player earnings for ranks 200-500
        start = 0
        end = 500
        if year in [2001, 2000, 1999, 1998]:
            if year == 2001:
                end = 400
            elif year == 2000:
                end = 300
            elif year == 1999 or year == 1998:
                end = 100
        for rank in range(start,end,100):
            if rank == 0:
                rank = ''
            else:
                rank = f'_{rank}'
            session = requests.Session()
            url = f'https://www.esportsearnings.com/history/{year}/top_players{str(rank)}'
            df_top_players = pull_player_data(df_top_players, session url, year)
            
    df_top_players = df_top_players.reset_index(drop=True)
    
    df_top_players['rank'] = pd.to_numeric(df_top_players['rank'].str.replace('\.', '', regex=True), errors='coerce')
    df_top_players['winnings_year'] = pd.to_numeric(df_top_players['winnings_year'].str.replace('[$,]', '', regex=True), errors='coerce')
    df_top_players['winnings_overall'] = pd.to_numeric(df_top_players['winnings_overall'].str.replace('[$,]', '', regex=True), errors='coerce')
    df_top_players['pct_winnings_overall'] = pd.to_numeric(df_top_players['pct_winnings_overall'].str.replace('%', '', regex=True), errors='coerce')
    df_top_players['year'] = pd.to_numeric(df_top_players['year'], errors='coerce')
    
    return df_top_players

#### Generic method for top games/country/team tables

In [30]:
def pull_table_data(df, url, month, year):
    soup = None
    table = None
    
    while soup is None:    
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        
    table = soup.find("table", class_ = 'detail_list_table')
    if table is None:
        return df
    else:
        for tr in table.find('tbody'):
            row = []
            for td in tr:
                row.append(td.text.strip())
            row.append(month)
            row.append(year)

            df = pd.concat([df, pd.DataFrame([row], columns=df.columns)])
            
    return df

#### Top games info per month/year

In [42]:
#get top games data each month/year
def top_games_table():
    columns = ['rank', 'game', 'prize', 'players', 'tournaments', 'month', 'year']
    df_top_games = pd.DataFrame(columns=columns)

    for year in range(2022, 1997, -1):
        for month in range(1,13):
            if month < 10:
                month_url = f'0{month}'
            else:
                month_url = month

            url = f'https://www.esportsearnings.com/history/{year}-{month_url}/games'
            df_top_games = pull_table_data(df_top_games, url, month_url, year)

    df_top_games = df_top_games.reset_index(drop=True)
    df_top_games['rank'] = pd.to_numeric(df_top_games['rank'].str.replace('\.', '', regex=True), errors='coerce')
    df_top_games['prize'] = pd.to_numeric(df_top_games['prize'].str.replace('[$,]', '', regex=True), errors='coerce')
    df_top_games['players'] = pd.to_numeric(df_top_games['players'].str.replace(' Players| Player', '', regex=True))
    df_top_games['tournaments'] = pd.to_numeric(df_top_games['tournaments'].str.replace(' Tournaments| Tournament', '', regex=True))
    df_top_games['month'] = pd.to_numeric(df_top_games['month'], errors='coerce')
    df_top_games['year'] = pd.to_numeric(df_top_games['year'], errors='coerce')
    
    return df_top_games

#### Top countries winnings per year

In [23]:
#get present day top countries by prize winnings data
def top_countries_table():
    url = 'https://www.esportsearnings.com/countries'
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')

    tbody = soup.find("tbody")
    country_details = []

    for row in tbody:
        row_details = row.find_all('td', class_='format_cell')
        rank = row_details[0].text
        country = row_details[1].text.strip()
        total_prize = row_details[2].text
        players = row_details[3].text
        top_game = row_details[4].text
        prize_top_game = row_details[5].text
        pct_total_prize = row_details[6].text
        row_list = [rank, country, total_prize, players, top_game, prize_top_game, pct_total_prize]
        country_details.append(row_list)

    df_top_countries = pd.DataFrame(country_details, columns=['rank', 'country', 'prize', 'players', 'top_game', 'prize_top_game', 'pct_total_prize'])
    
    df_top_countries['rank'] = pd.to_numeric(df_top_countries['rank'].str.replace('\.', '', regex=True), errors='coerce')
    df_top_countries['prize'] = pd.to_numeric(df_top_countries['prize'].str.replace('[$,]', '', regex=True), errors='coerce')
    df_top_countries['players'] = df_top_countries['players'].str.replace(r',', '', regex=True)
    df_top_countries['players'] = pd.to_numeric(df_top_countries['players'].str.replace(r' Players| Player', '', regex=True))
    df_top_countries['prize_top_game'] = pd.to_numeric(df_top_countries['prize_top_game'].str.replace('[$,]', '', regex=True), errors='coerce')
    df_top_countries['pct_total_prize'] = pd.to_numeric(df_top_countries['pct_total_prize'].str.replace('%', '', regex=True), errors='coerce')
    
    return df_top_countries

In [39]:
#get top countries data each month/year
def top_countries_history_table():
    columns = ['rank', 'country', 'prize', 'players', 'month', 'year']
    df_top_countries_history = pd.DataFrame(columns=columns)
    
    for year in range(2022, 1997, -1):
        for month in range(1,13):
            if month < 10:
                month_url = f'0{month}'
            else:
                month_url = month

            url = f'https://www.esportsearnings.com/history/{year}-{month_url}/countries'
            df_top_countries_history = pull_table_data(df_top_countries_history, url, month_url, year)
        
    df_top_countries_history['rank'] = pd.to_numeric(df_top_countries_history['rank'].str.replace('\.', '', regex=True), errors='coerce')
    df_top_countries_history['prize'] = pd.to_numeric(df_top_countries_history['prize'].str.replace('[$,]', '', regex=True), errors='coerce')
    df_top_countries_history['players'] = pd.to_numeric(df_top_countries_history['players'].str.replace(r' Players| Player', '', regex=True))
    df_top_countries_history['month'] = pd.to_numeric(df_top_countries_history['month'], errors='coerce')
    df_top_countries_history['year'] = pd.to_numeric(df_top_countries_history['year'], errors='coerce')
    
    return df_top_countries_history

#### Top Teams by winnings per year

In [53]:
#get top teams data each month/year
def top_teams_history_table():
    columns = ['rank', 'team', 'prize', 'tournaments', 'month', 'year']
    df_top_teams = pd.DataFrame(columns=columns)
    
    for year in range(2022, 1997, -1):
        for month in range(1,13):
            if month < 10:
                month_url = f'0{month}'
            else:
                month_url = month

            url = f'https://www.esportsearnings.com/history/{year}-{month_url}/teams'
            df_top_teams = pull_table_data(df_top_teams, url, month_url, year)
        
    df_top_teams['rank'] = pd.to_numeric(df_top_teams['rank'].str.replace('\.', '', regex=True), errors='coerce')
    df_top_teams['prize'] = pd.to_numeric(df_top_teams['prize'].str.replace('[$,]', '', regex=True), errors='coerce')
    df_top_teams['tournaments'] = pd.to_numeric(df_top_teams['tournaments'].str.replace(' Tournaments| Tournament', '', regex=True))
    df_top_teams['month'] = pd.to_numeric(df_top_teams['month'], errors='coerce')
    df_top_teams['year'] = pd.to_numeric(df_top_teams['year'], errors='coerce')
    
    return df_top_teams

#### Player profile history

In [55]:
#get player's annual history of prize winnings/tournament count
def pull_player_yearly_history(player_profile, df_players_yearly_history):

    table = None
    while table is None:
        url = f'https://www.esportsearnings.com{player_profile}'
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find("table", class_ = 'detail_list_table player_summary_table')

    tr_list = table.find_all('tr', class_='format_row highlight')
    year_history = []

    for tr in tr_list:
        row = tr.find_all('td')
        
        year = row[0].text
        prize = row[1].text
        tournaments = row[2].text
        pct_total = row[3].text

        row_year = [year, prize, tournaments, pct_total, player_profile]
        year_history.append(row_year)
        print(f'year: {year}')
        print(f'prize: {prize}')
        print(f'tournaments: {tournaments}')   
        print(f'pct_total: {pct_total}')
        print(f'profile_link: {player_profile}')
        print('---------------------------------')
    
    df_players_yearly_history = pd.concat([df_players_yearly_history, pd.DataFrame(year_history, columns=df_players_yearly_history.columns)])

    return df_players_yearly_history

In [56]:
#pull every unique player's annual history of prize winnings/tournament count
def players_yearly_history_table(player_profile_list):
    df_players_yearly_history = pd.DataFrame(columns=['year', 'prize', 'tournaments', 'pct_total', 'profile_link'])

    for player in player_profile_list:
        df_players_yearly_history = pull_player_yearly_history(player, df_players_yearly_history)
    
    df_players_yearly_history = df_players_yearly_history.reset_index(drop=True)
    df_players_yearly_history = df_players_yearly_history.replace('NaN', np.nan)
    
    df_players_yearly_history['year'] = pd.to_numeric(df_players_yearly_history['year'], errors='coerce')
    df_players_yearly_history['prize'] = pd.to_numeric(df_players_yearly_history['prize'].str.replace('[$,]', '', regex=True), errors='coerce')
    df_players_yearly_history['tournaments'] = pd.to_numeric(df_players_yearly_history['tournaments'].str.replace(' Tournaments| Tournament', '', regex=True))
    df_players_yearly_history['pct_total'] = pd.to_numeric(df_players_yearly_history['pct_total'].str.replace('%', '', regex=True), errors='coerce')
    
    return df_players_yearly_history

#### Player Profile

In [12]:
#get player demographic and game data
def pull_player_profile(player_profile, df_player_info):
    
    player_info = None
    
    while player_info is None:
        url = f'https://www.esportsearnings.com{player_profile}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        player_info = soup.find("div", class_='info_box')

    #get player alias
    player_alias = player_info.find('h1', class_='info_box_title')
    
    game = re.split('\- ', player_alias.text)[-1]
    game = re.split(' Player', game)[0]
    
    alias = re.split(f'\- {game}', player_alias.text)[0].strip()
    
    if len(re.split('"(.*)"', alias)) == 1:
        name = re.split('"(.*)"', alias)[0]
        player_id = 'NaN'
    else:
        first = re.split('"(.*)"', alias)[0].strip()
        last = re.split('"(.*)"', alias)[2].strip()
        player_id = re.split('"(.*)"', alias)[1].strip()
        
        if first == '-' or last == '-':
            name = 'NaN'
        else:
            name = f'{first} {last}'

    #get player age
    player_info_detailed = player_info.find_all('div', class_='format_cell')

    if player_info_detailed[4].text == 'Age:':
        age = player_info_detailed[5].text
    else:
        age = 'NaN'

    #get player country
    country = player_info.find('div', class_='info_country').text
    
       
    print(f'name: {name}')
    print(f'player_id: {player_id}')
    print(f'game: {game}')
    print(f'age: {age}')
    print(f'country: {country}')
    print(f'profile_link: {player_profile}')
    print('-------------------------------------')
    
    player_info_data = [name, player_id, game, age, country, player_profile]
    df_player_info = pd.concat([df_player_info, pd.DataFrame([player_info_data], columns=df_player_info.columns)])

    return df_player_info

In [11]:
#get all unique players demographic and game data
def player_profile_table(player_profile_list):
    df_player_info = pd.DataFrame(columns=['name', 'player_id', 'game', 'age', 'country', 'profile_link'])
    
    for player in player_profile_list:
        df_player_info = pull_player_profile(player, df_player_info)
    
    df_player_info = df_player_info.reset_index(drop=True)
    df_player_info = df_player_info.replace('NaN', np.nan)
    df_player_info['age'] = pd.to_numeric(df_player_info['age'], errors='coerce', downcast='integer')
    
    return df_player_info

### esport games details

In [46]:
#for pulling game and genre data
def esports_games_table():
    url = 'https://www.esportsearnings.com/games/browse-by-genre'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    game_info = soup.find("div", class_='detail_list_box')
    
    genre_strings = game_info.find_all("span", class_='games_main_genre_title')
    genre_games = game_info.find_all('div', class_="games_main_genre_body")
    
    df_esport_games = pd.DataFrame(columns=['title', 'genre', 'prize', 'players' ,'tournaments'])
    
    for i, genre_title in enumerate(genre_strings):
        genre = genre_title.text
        game_details = []

        for game in genre_games[i]:
            if game.div is None:
                continue
            else:
                game_title = game.div.text
                game_stats = game.find('div', class_='games_main_game_stats')
                prize = game_stats.contents[0]
                players = game_stats.contents[2]
                tournaments = game_stats.contents[4]

                row = [game_title, genre, prize, players, tournaments]
                game_details.append(row)

        df_esport_games = pd.concat([df_esport_games, pd.DataFrame(game_details, columns=df_esport_games.columns)])

    df_esport_games = df_esport_games.reset_index(drop=True)
    
    df_esport_games['prize'] = pd.to_numeric(df_esport_games['prize'].str.replace('[$,]', '', regex=True), errors='coerce')
    df_esport_games['players'] = pd.to_numeric(df_esport_games['players'].str.replace(' Players| Player', '', regex=True))
    df_esport_games['tournaments'] = pd.to_numeric(df_esport_games['tournaments'].str.replace(' Tournaments| Tournament', '', regex=True))
    
    return df_esport_games

In [ ]:
#scrape top players
df_top_players = top_players_table()
df_top_players.info()

#get only unique players as they can appear in the top 500 players multiple times between 1998-2022
unique_profiles = df_top_players[~df_top_players['player_profile'].duplicated()]['player_profile']

#scrape top games
df_top_games = top_games_table()
df_top_games.info()

#scrape top countries
df_top_countries = top_countries_table()
df_top_countries.info()

#scrape top countries history
df_top_countries_history = top_countries_history_table()
df_top_countries_history.info()

#scrape top teams
df_top_teams = top_teams_history_table()
df_top_teams.info()

#scrape player's yearly history
#use unique_profiles derived from top_players table
df_players_yearly_history = players_yearly_history_table(unique_profiles)
df_players_yearly_history.info()

#scrape player's demographic info
df_player_info = player_profile_table(unique_profiles)
df_player_info.info()

#scrape all genres/games data
df_esport_games = esports_games_table()
df_esport_games.info()

#save to .csv file to load into SQL to query for Tableau visualisation
df_top_players.to_csv('to_sql/top_players.csv', index=False)
df_top_games.to_csv('to_sql/top_games.csv', index=False)
df_top_countries.to_csv('to_sql/top_countries.csv', index=False)
df_top_countries_history.to_csv('to_sql/top_countries_history.csv', index=False)
df_top_teams.to_csv('to_sql/top_teams.csv', index=False)
df_players_yearly_history.to_csv('to_sql/top_player_history.csv', index=False)
df_player_info.to_csv('to_sql/player_profiles.csv', index=False)
df_esport_games.to_csv('to_sql/all_esport_games.csv', index=False)